In [1]:
import gzip
import json
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from scipy.sparse import lil_matrix
from torch_geometric.utils.convert import from_scipy_sparse_matrix
import torch
from pandas import json_normalize
from IPython.display import display

In [2]:
print(torch.cuda.is_available())
device = torch.device('cuda')

False


In [3]:
with gzip.open('../data/RosettaStone-GraphData-2023-09-22/superblue1/1/superblue1.json.gz','rb') as f:
    design = json.loads(f.read().decode('utf-8'))
    
instances = pd.DataFrame(design['instances'])
nets = pd.DataFrame(design['nets'])

conn=np.load('../data/RosettaStone-GraphData-2023-09-22/superblue1/superblue1_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])

with gzip.open('../data/RosettaStone-GraphData-2023-09-22/cells.json.gz', 'rb') as f:
    cells_json = json.loads(f.read())
cells = pd.DataFrame(cells_json).drop(columns=['name', 'width', 'height']).set_index('id')

In [4]:
cells['terms'] = cells['terms'].apply(lambda x: pd.DataFrame(x))
cells

,terms
id,
0,name id dir xloc yloc 0 A 1 0 ...
1,name id dir xloc yloc 0 A 1 0 ...
2,name id dir xloc yloc 0 A 1 0 ...
3,name id dir xloc yloc 0 D 1 0...
4,name id dir xloc yloc 0 A 1 0 ...
...,...
30131,Empty DataFrame Columns: [] Index: []
30132,Empty DataFrame Columns: [] Index: []
30133,Empty DataFrame Columns: [] Index: []


In [5]:
instances.head()

,name,id,xloc,yloc,cell,orient
0,o0,0,3567360,5044950,132,0
1,o1,1,2863200,5351310,359,6
2,o10,2,2846400,5351310,176,0
3,o100,3,2508000,5744250,224,0
4,o1000,4,2479680,5757570,360,0


In [6]:
A_tensor = torch.sparse_coo_tensor(
    torch.FloatTensor([A.row, A.col]),
    torch.Tensor(A.data)
).coalesce()
#A_tensor.to(device)

/var/folders/6b/6xx67mh125s3q_lscfz571ym0000gn/T/ipykernel_10188/825442066.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  torch.FloatTensor([A.row, A.col]),


In [7]:
bad = []
global bad

In [8]:
def direct(c, val):
    '''
    params:
    c: column index / terminal
    val: tensor value for lookup
    '''
    current_cell = cells['terms'][instances.iloc[c]['cell']]
    
    if val > current_cell.shape[0] - 1:
        bad.append((c, val))
        
    direction = current_cell.iloc[min(val, current_cell.shape[0] - 1)]['dir']
    if direction == 1:
        return 0
    return 1

In [9]:
# A_tensor.indices()[1] is columns of terminal matrix, or gates
indices = A_tensor.indices()
terminal_values = (A_tensor.values()).clone().detach()
gate_indices = indices[0]

In [10]:
out = lil_matrix((A_tensor.values().shape[0], 1))

In [11]:
i = 0
for gate_index, val_i in zip(gate_indices, range(terminal_values.shape[0])):
#     if i % 196595 == 0:
#         print(i)
   # i += 1
    out[val_i] = direct(c=int(gate_index.item()), val=int(terminal_values[val_i].item()) - 1)

In [12]:
differences = []
for b in bad:
    diff = b[1] - (cells['terms'][instances.iloc[b[0]]['cell']].shape[0] - 1) # .iloc[b[1]]['dir']
    differences.append(diff)
differences[0:10]

[2, 1, 3, 1, 3, 3, 3, 3, 3, 1]

In [101]:
out_tensor = torch.tensor(out.T.toarray())[0]

In [105]:
out_tensor

tensor([1., 1., 1.,  ..., 1., 1., 0.], dtype=torch.float64)

In [106]:
A_tensor.values().copy_(out_tensor)

tensor([1., 1., 1.,  ..., 1., 1., 0.])

In [109]:
 A_tensor

tensor(indices=tensor([[     0,      0,      0,  ..., 797937, 797937, 797937],
                       [613721, 613832, 613943,  ...,      0, 756354, 822927]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 0.]),
       size=(797938, 823024), nnz=2948927, layout=torch.sparse_coo)

In [ ]:
A = torch.sparse.mm(A_tensor, A_tensor.T)

In [ ]:
A